In [1]:
from Bio import AlignIO

import pandas as pd
from os.path import abspath, join
import copy

from skbio import TreeNode

In [41]:
bacteria="Escherichia_coli"
marker_threshold = "50"
sample_group="" ##mammallian, nonMammallian ... 
bacteria_trimmed =bacteria[0]+bacteria[bacteria.find("_"):len(bacteria)]
output_dir=join(tree_dir,'output_'+marker_threshold+"_"+bacteria_trimmed + sample_group)
output_dir

'/panfs/panfs1.ucsd.edu/panscratch/jhc103/VertMetaphlan-frmerged/strain-trees-output/output_50_E_coli'

In [19]:
## Read in metadata
md_dir = '/panfs/panfs1.ucsd.edu/panscratch/jhc103/VertMetaphlan-frmerged/metadata'
host_md_fp = join(md_dir, 'vert_metadata_8_23.txt')
host_md_orig = pd.read_csv(host_md_fp, sep='\t', encoding='windows-1252')

## various paths
tree_dir = abspath('/panfs/panfs1.ucsd.edu/panscratch/jhc103/VertMetaphlan-frmerged/strain-trees-output/')

In [30]:
## Read in alignment
align = AlignIO.read(join(output_dir,"s__"+bacteria+".StrainPhlAn3_concatenated.aln"),"fasta")

In [31]:
## Change value of alignment
for value in align:
    if len(host_md_orig.loc[host_md_orig["SampleID2"]==value.id,"reads_per_file"])==0: ## for reference genome 
        value.id = "ref_seq"
    else:
        value.id=str(int(host_md_orig.loc[host_md_orig["SampleID2"]==value.id,"reads_per_file"]))

In [32]:
## Write new alignment 
count = AlignIO.write(align,join(output_dir,"align.phy"),"phylip")

In [33]:
## Read in tree
tree_parse= TreeNode.read(join(output_dir,"RAxML_parsimonyTree.s__"+bacteria+".StrainPhlAn3.tre"), format='newick', convert_underscores=False)

In [34]:
## Change tip name
for node in tree_parse.tips():
    if len(host_md_orig.loc[host_md_orig["SampleID2"]==node.name,"reads_per_file"])==0: ## for reference genome 
        node.name = "ref_seq"
    else:
        node.name=str(int(host_md_orig.loc[host_md_orig["SampleID2"]==node.name,"reads_per_file"]))

In [42]:
## Write tree to file
tree_parse.write(join(output_dir,"tree_parsimony_tips_changed.nwk"), format="newick")

'/panfs/panfs1.ucsd.edu/panscratch/jhc103/VertMetaphlan-frmerged/strain-trees-output/output_50_E_coli/tree_parsimony_tips_changed.nwk'

In [44]:
!cd $output_dir
!ls $output_dir
!ls $output_dir/align.phy

align.phy
Escherichia_coli_sample_marker_percent.tsv
GAMMA
RAxML_bestTree.s__Escherichia_coli.StrainPhlAn3.tre
RAxML_info.s__Escherichia_coli.StrainPhlAn3.tre
RAxML_log.s__Escherichia_coli.StrainPhlAn3.tre
RAxML_parsimonyTree.s__Escherichia_coli.StrainPhlAn3.tre
RAxML_result.s__Escherichia_coli.StrainPhlAn3.tre
s__Escherichia_coli.info
s__Escherichia_coli.polymorphic
s__Escherichia_coli.StrainPhlAn3_concatenated.aln
tree_parsimony_tips_changed.nwk
/panfs/panfs1.ucsd.edu/panscratch/jhc103/VertMetaphlan-frmerged/strain-trees-output/output_50_E_coli/align.phy


In [47]:
!/home/jhc103/miniconda3/envs/mpa3-py37/bin/raxmlHPC-PTHREADS-SSE3 -T 16 -f x -m GTRGAMMA -s $output_dir/align.phy -t $output_dir/tree_parsimony_tips_changed.nwk -n GTRGAMMA_dist -w $output_dir

Warning, you specified a working directory via "-w"
Keep in mind that RAxML only accepts absolute path names, not relative ones!


Using BFGS method to optimize GTR rate parameters, to disable this specify "--no-bfgs" 


This is the RAxML Master Pthread

This is RAxML Worker Pthread Number: 2

This is RAxML Worker Pthread Number: 3

This is RAxML Worker Pthread Number: 1

This is RAxML Worker Pthread Number: 4

This is RAxML Worker Pthread Number: 5

This is RAxML Worker Pthread Number: 11

This is RAxML Worker Pthread Number: 15

This is RAxML Worker Pthread Number: 9

This is RAxML Worker Pthread Number: 10

This is RAxML Worker Pthread Number: 8

This is RAxML Worker Pthread Number: 6

This is RAxML Worker Pthread Number: 7

This is RAxML Worker Pthread Number: 12

This is RAxML Worker Pthread Number: 13

This is RAxML Worker Pthread Number: 14


This is RAxML version 8.2.12 released by Alexandros Stamatakis on May 2018.

With greatly appreciated code contributions by:
Andre Aberer 

In [46]:
!!/home/jhc103/miniconda3/envs/mpa3-py37/bin/raxmlHPC-PTHREADS-SSE3 -h

['',
 '',
 'This is RAxML version 8.2.12 released by Alexandros Stamatakis on May 2018.',
 '',
 'With greatly appreciated code contributions by:',
 'Andre Aberer      (HITS)',
 'Simon Berger      (HITS)',
 'Alexey Kozlov     (HITS)',
 'Kassian Kobert    (HITS)',
 'David Dao         (KIT and HITS)',
 'Sarah Lutteropp   (KIT and HITS)',
 'Nick Pattengale   (Sandia)',
 'Wayne Pfeiffer    (SDSC)',
 'Akifumi S. Tanabe (NRIFS)',
 'Charlie Taylor    (UF)',
 '',
 '',
 'Please also consult the RAxML-manual',
 '',
 'Please report bugs via the RAxML google group!',
 'Please send us all input files, the exact invocation, details of the HW and operating system,',
 'as well as all error messages printed to screen.',
 '',
 '',
 'raxmlHPC[-SSE3|-AVX|-PTHREADS|-PTHREADS-SSE3|-PTHREADS-AVX|-HYBRID|-HYBRID-SSE3|HYBRID-AVX]',
 '      -s sequenceFileName -n outputFileName -m substitutionModel',
 '      [-a weightFileName] [-A secondaryStructureSubstModel]',
 '      [-b bootstrapRandomNumberSeed] [-B wcCrit